In [1]:
print("""\
# *************************************************************************** #
# *************************************************************************** #
# TP2 : ALGEBRE LINEAIRE SUR UN ANNEAU PRINCIPAL                              #
# *************************************************************************** #
# *************************************************************************** #
""")

# CONSIGNES
#
# Les seules lignes a modifier sont annoncee par "Code pour l'exercice"
# indique en commmentaire et son signalees
# Ne changez pas le nom des variables
#
# CONSEILS
#
# Ce modele vous sert a restituer votre travail. Il est deconseille d'ecrire
# une longue suite d'instruction et de debugger ensuite. Il vaut mieux tester
# le code que vous produisez ligne apres ligne, afficher les resultats et
# controler que les objets que vous definissez sont bien ceux que vous attendez.
#
# Vous devez verifier votre code en le testant, y compris par des exemples que
# vous aurez fabrique vous-meme.
#


reset()
print("""\
# ****************************************************************************
# MISE SOUS FORME NORMALE D'HERMITE
# ****************************************************************************
""")

# Donnees de l'enonce de l'exercice

A = matrix(ZZ,[
        [-2,  3,  3,  1],
        [ 2, -1,  1, -3],
        [-4,  0, -1, -4]])

A1 = random_matrix(ZZ, 7, 8, algorithm='echelonizable', rank=3)

U = identity_matrix(4)

# Code pour l'EXERCICE

def cherche_pivot_non_nul(A, U, i, j):
    """
    Echange la colonne j avec une colonne à gauche pour que A[i,j] soit non nul.
    """
    """
    The function aims to find the rightmost column (denoted as j0)
     in the submatrix A[:, j:] (columns from j to the end) 
     such that the element at position (i, j0) is non-zero. 

    If such a column is found (i.e., j0 != -1), 
    it swaps columns j and j0 in both matrices A and U. 
    The function returns True if a pivot is found and False otherwise.

    In the context of Hermite Normal Form computations, 
    this function is likely used to ensure that 
    the leading entry in the current row (specified by the index i) is non-zero, 
    facilitating subsequent operations in the reduction process.
    """
    j0 = -1
    for k in range(j,-1,-1):
        if A[i,k]!=0:
            j0 = k
            break
    
    if j0 != -1:
       A.swap_columns(j, j0)
       U.swap_columns(j, j0)
       return True #returns True if a pivot has been found
    else:
        return False
    

def normalise_pivot(A, U, i, j):
    """
    Multiplie la colonne j si besoin pour que A[i,j] soit positif.
    """
    """
    The purpose of this function is to ensure that the element at position (i,j)
    un matrix A is positive.
    This function checks if the element at position (i,j) in matrix A
    is negative. The function multiplies the entire column j by -1 to make it positive.
    """
    if A[i,j] < 0:
        A[:,j] = A[:,j] * -1
        U[:,j] = U[:,j] * -1

def annule_a_gauche(A, U, i, j):
    """
    Annule les coefficients à gauche de A[i,j]
    Eliminate the coefficients to the left of the element A[i,j]
    in matrix A and U.
    it performs operations to eliminate the coefficient 
    at position (i, k) using Bézout's relations. 
    """
    for k in range(j-1, -1, -1):
        a = A[i,j]
        b = A[i,k]
        d, u, v = xgcd(a, b) #Eucidean
        A[:,j], A[:,k] = u * A[:,j] + v * A[:,k], -(b/d) * A[:,j] + (a/d) * A[:,k]
        U[:,j], U[:,k] = u * U[:,j] + v * U[:,k], -(b/d) * U[:,j] + (a/d) * U[:,k]

def reduit_a_droite(A,U,i,j):
    """
    Réduit les coefficients à gauche de A[i,j] modulo A[i,j]
    subtracting a multiple of the j-th column from the k-th column 
    to make the coefficient at (i,k) modulo 'A[i,j]'
    """
    for k in range(j+1, A.ncols()):
        quotient  = A[i,k]// A[i,j]
        A[:,k] = A[:,k] - quotient * A[:,j]
        U[:,k] = U[:,k] - quotient * U[:,j]



def MyHNF(A):
    """
    Forme normale d'Hermite selon votre code
    """
    m = A.nrows()
    n = A.ncols()
    H = copy(A)
    U = identity_matrix(n)
    l = 0 # initialize a variable l to 0
    i = m-1 # start from the bottom row
    k = n-1 # start from the rightmost column

    while i>= l:
        # Check if there is a non-zero pivot in the current column
        if cherche_pivot_non_nul(H, U, i, k):
            # If a pivot is found, perform the following operations:
            normalise_pivot(H,U,i,k) #Normalize the pivot element
            annule_a_gauche(H,U,i,k) #Make entries above the pivot zero
            reduit_a_droite(H,U,i,k) #Make entries to the right of the pivot zero
        
            i = i-1
            k = k-1
        else:
            i = i-1
    # Check if HNF computation is successful
    assert(H-A*U==0)
    # Return the computed Hermite Normal Form (H) and the transformation matrix (U)
    return H,U

def SageHNF(A):
    """
    Forme normale d'Hermite de SAGE avec la convention francaise :
    Les vecteurs sont ecrits en colonne.
    """
    m = A.nrows()
    n = A.ncols()
    Mm = identity_matrix(ZZ,m)[::-1,:]
    Mn = identity_matrix(ZZ,n)[::-1,:]
    AA = (Mm*A).transpose()
    HH, UU = AA.hermite_form(transformation=True)
    H = (HH*Mm).transpose()*Mn
    U = UU.transpose()*Mn
    assert(H-A*U==0)
    return H,U

H,  U  = MyHNF(A)
HH, UU = SageHNF(A)

# test number matrix
def test(number):
    for i in range(number):
        n = ZZ.random_element(20)
        m = ZZ.random_element(20)

        A = random_matrix(ZZ,n,m)
        H, U = MyHNF(A)
        HH, UU = SageHNF(A)
        if HH!=H:
            return False
    return True

test = test(10)

# # Affichage des resultats

print("\n$ Question 4")
print("La matrice A = ")
print(A)
print("a pour forme normale d'Hermite H=")
print(H)
print("et matrice de transformation U=")
print(U)
print("\n$ Question 5")
print("D'apres SageMath, la matrice A a pour forme normale d'Hermite H=")
print(HH)
print("et matrice de transformation U=")
print(UU)
print("\n$ Question 6")
print("Les deux fonctions coincident-elles sur une centaine d'exemples ?")
print(test)

# *************************************************************************** #
# *************************************************************************** #
# TP2 : ALGEBRE LINEAIRE SUR UN ANNEAU PRINCIPAL                              #
# *************************************************************************** #
# *************************************************************************** #

# ****************************************************************************
# MISE SOUS FORME NORMALE D'HERMITE
# ****************************************************************************


$ Question 4
La matrice A = 
[-2  3  3  1]
[ 2 -1  1 -3]
[-4  0 -1 -4]
a pour forme normale d'Hermite H=
[0 4 1 2]
[0 0 1 0]
[0 0 0 1]
et matrice de transformation U=
[  8   3   3   6]
[ 19   8   7  15]
[-12  -4  -4  -9]
[ -5  -2  -2  -4]

$ Question 5
D'apres SageMath, la matrice A a pour forme normale d'Hermite H=
[0 4 1 2]
[0 0 1 0]
[0 0 0 1]
et matrice de transformation U=
[  8   3   3   6]
[ 19 

In [2]:
reset()
print("""\
# ****************************************************************************
# MISE SOUS FORME NORMALE DE SMITH
# ****************************************************************************
""")

# Donnees de l'enonce de l'exercice

X1 = matrix(ZZ,2,3,[
        [4, 7, 2],
        [2, 4, 6]])

X2 = matrix(ZZ,3,3,[
        [-397, 423, 352],
        [   2,  -3,   1],
        [-146, 156, 128],
])

PolQ.<xQ> = PolynomialRing(QQ)
AQ = matrix(PolQ,3,[
            [xQ + 1,  2,     -6],
            [     1, xQ,     -3],
            [     1,  1, xQ - 4]])
Pol2.<x2> = PolynomialRing(FiniteField(2))
AF2 = matrix(Pol2,3,[
            [x2 + 1,  2,     -6],
            [     1, x2,     -3],
            [     1,  1, x2 - 4]])
Pol3.<x3> = PolynomialRing(FiniteField(3))
AF3 = matrix(Pol3,3,[
            [x3 + 1,  2,     -6],
            [     1, x3,     -3],
            [     1,  1, x3 - 4]])
Pol5.<x5> = PolynomialRing(FiniteField(5))
AF5 = matrix(Pol5,3,[
            [x5 + 1,  2,     -6],
            [     1, x5,     -3],
            [     1,  1, x5 - 4]])

# Code pour l'EXERCICE

def cherche_pivot_et_echange(A,L,U,k):
    for i in range(k, A.nrows()):
        for j in range(k, A.ncols()):
            if A[i,j]!=0 :
                A.swap_rows(k,i)
                L.swap_rows(k,i)
                A.swap_columns(k,j)
                U.swap_columns(k,j)
                return True
    return False


def test_arret(A,k):
    for i in range(k+1, A.nrows()):
        if A[i,k]!=0:
            return False
    for j in range(k+1, A.ncols()):
        if A[k,j]!=0:
            return False
    return True

def bezout_lignes(A,L,k):
    for i in range(k+1, A.nrows()):
        d,s,t = xgcd(A[k,k],A[i,k])
        if A[i,k] % A[k,k] == 0:
            d = A[k,k]
            s = 1
            t = 0
        u, v = -A[i,k]/d, A[k,k]/d
        A[k,:], A[i,:] = s*A[k,:] + t*A[i,:], u*A[k,:] + v*A[i,:]
        L[k,:], L[i,:] = s*L[k,:] + t*L[i,:], u*L[k,:] + v*L[i,:]

def bezout_colonnes(A,U,k):
    for j in range(k+1,A.ncols()):
        d, s, t = xgcd(A[k,k], A[k,j])
        if A[k,j] % A[k,k] == 0:
            d = A[k,k]
            s = 1
            t = 0
        u, v = -A[k,j]/d, A[k,k]/d
        A[:,k], A[:,j] = s*A[:,k] + t*A[:,j], u*A[:,k] + v*A[:,j]
        U[:,k], U[:,j] = s*U[:,k] + t*U[:,j], u*U[:,k] + v*U[:,j]

def test_divisibilite(A,U,k):
    for i in range(k+1, A.nrows()):
        for j in range(k+1, A.ncols()):
            if A[i,j] % A[k,k] != 0 :
                A[:,k] = A[:,k] + A[:,j]
                U[:,k] = U[:,k] + U[:,j]
                return

def MySNF(A):
    """
    Forme normale de Smith selon votre code
    """
    m = A.nrows()
    n = A.ncols()
    H = copy(A)
    L = identity_matrix(A.base_ring(),m)
    U = identity_matrix(A.base_ring(),n)
    # ECRIVEZ VOTRE CODE ICI
    # Why we write like this? Fucking shit!
    for k in range(min(n, m)):
        if H[k, k] == 0:
            if not cherche_pivot_et_echange(H, L, U, k):
                return H, L, U
        while not test_arret(H, k):
            bezout_lignes(H, L, k)
            bezout_colonnes(H, U, k)
            test_divisibilite(H, U, k)

    assert (H-L*A*U==0)
    return H,L,U

H1, L1, U1 = MySNF(X1)
H2, L2, U2 = MySNF(X2)

HQ, _, _ = MySNF(AQ)
HF2, _, _ = MySNF(AF2)
HF3, _, _ = MySNF(AF3)
HF5, _, _ = MySNF(AF5)

def test(nb):
    for i in range(nb):
        n = ZZ.random_element(5)
        m = ZZ.random_element(5)

        A = random_matrix(ZZ,n,m)
        H, _, _ = MySNF(A)
        HH, _, _ = A.smith_form()
        print(f"my solution of smith form of A {H}\n")
        print(f"smith form of A ,{HH}\n")
        for k in range(min(H.nrows(), H.ncols())):
            if abs(H[k,k]) != HH[k,k]:
                return False
    return True
    
test = test(10)

# # Affichage des resultats

print("\n$ Question 4")
print("La matrice X1 = ")
print(X1)
print("a pour forme normale de Smith H1=")
print(H1)
print("et matrice de transformation L1=")
print(L1)
print("et matrice de transformation U1=")
print(U1)
print("La matrice X2 = ")
print(X2)
print("a pour forme normale de Smith H2=")
print(H2)
print("et matrice de transformation L2=")
print(L2)
print("et matrice de transformation U2=")
print(U2)

print("\n$ Question 5")
print("La forme normale de Smith sur Q est ")
print(AQ)
print("La forme normale de Smith sur F2 est ")
print(AF2)
print("La forme normale de Smith sur F3 est ")
print(AF3)
print("La forme normale de Smith sur F5 est ")
print(AF5)

print("\n$ Question 6")
print("Votre fonction coincide avec celle de Sage ?")
print(test)

# ****************************************************************************
# MISE SOUS FORME NORMALE DE SMITH
# ****************************************************************************

my solution of smith form of A []

smith form of A ,[]

my solution of smith form of A [1 0]
[0 1]
[0 0]

smith form of A ,[1 0]
[0 1]
[0 0]

my solution of smith form of A [-1]
[ 0]
[ 0]

smith form of A ,[1]
[0]
[0]

my solution of smith form of A []

smith form of A ,[]

my solution of smith form of A [0 0]

smith form of A ,[0 0]

my solution of smith form of A []

smith form of A ,[]

my solution of smith form of A [-1  0  0  0]
[ 0  1  0  0]
[ 0  0  1  0]

smith form of A ,[1 0 0 0]
[0 1 0 0]
[0 0 1 0]

my solution of smith form of A [-1]
[ 0]
[ 0]
[ 0]

smith form of A ,[1]
[0]
[0]
[0]

my solution of smith form of A []

smith form of A ,[]

my solution of smith form of A [1]
[0]
[0]
[0]

smith form of A ,[1]
[0]
[0]
[0]


$ Question 4
La matrice X1 = 
[4 7 2]
[2 4 6]
a pour forme normale

In [6]:
reset()
print("""\
# ****************************************************************************
# RESOLUTION DE SYSTEME LINEAIRE HOMOGENE
# ****************************************************************************
""")
#Use the smith form to solve this problem
# Donnees de l'enonce de l'exercice

def SageHNF(A):
    """
    Forme normale d'Hermite de SAGE avec la convention francaise :
    Les vecteurs sont ecrits en colonne.
    """
    m = A.nrows()
    n = A.ncols()
    Mm = identity_matrix(ZZ,m)[::-1,:]
    Mn = identity_matrix(ZZ,n)[::-1,:]
    AA = (Mm*A).transpose()
    HH, UU = AA.hermite_form(transformation=True)
    H = (HH*Mm).transpose()*Mn
    U = UU.transpose()*Mn
    assert(H-A*U==0)
    return H,U

X = matrix(ZZ,[
      [ -2,  3,  3,  1],
      [  2, -1,  1, -3],
      [ -4,  0, -1, -4]])

# Code pour l'EXERCICE
H, U = SageHNF(X)
row = 0
while row<H.ncols() and H[:,row] == 0:
    row+=1
L =[U[:,i] for i in range(row)] # liste des vecteurs d'une base

# # Affichage des resultats

print("Le systeme a pour racine le module engendre par")
print(L)

# ****************************************************************************
# RESOLUTION DE SYSTEME LINEAIRE HOMOGENE
# ****************************************************************************

Le systeme a pour racine le module engendre par
[[  8]
[ 19]
[-12]
[ -5]]


print("""\
# ****************************************************************************
# IMAGE D'UNE MATRICE
# ****************************************************************************
""")

# Donnees de l'enonce de l'exercice

A  = matrix(ZZ, [
           [ 15,  8, -9, 23,  -9],
           [ 22, 22,  7, -8,  20],
           [ 21, 18, -1, -7,  -3],
           [  3, -1,  0, 12, -16]])


# Code pour l'EXERCICE

I = identity_matix()

test = false # test a ecrire

# # Affichage des resultats

print("L'image de")
print(A)
print("est-elle egale a ZZ^4 ?")
print(test)

In [17]:
# Given matrix A
A = matrix(ZZ, [
    [15,  8, -9, 23, -9],
    [22, 22,  7, -8, 20],
    [21, 18, -1, -7, -3],
    [3,  -1,  0, 12, -16]
])

H,_,_ = A.smith_form()
r = 0
while H[:,r] != 0 and r<H.ncols():
    r+=1

test = (r==4)

print("L'image de")
print(A)
print("est-elle egale a ZZ^4 ?")
print(test)

L'image de
[ 15   8  -9  23  -9]
[ 22  22   7  -8  20]
[ 21  18  -1  -7  -3]
[  3  -1   0  12 -16]
est-elle egale a ZZ^4 ?
True


In [20]:
reset()
print("""\
# ****************************************************************************
# RESOLUTION DE SYSTEME LINEAIRE NON-HOMOGENE
# ****************************************************************************
""")
# There seems to be a diffrences on the result
# Donnees de l'enonce de l'exercice

X1 = matrix(ZZ,[
           [ -6,  12,  6],
           [ 12, -16, -2],
           [ 12, -16, -2]])

b1 = vector(ZZ,[ -6, 4, 4])

PolF5.<x> = PolynomialRing(GF(5))

X2 = matrix(PolF5,[
           [ x + 1, 2,     4],
           [     1, x,     2],
           [     1, 1, x + 1]])

b2 = vector(PolF5,[ 3*x+2, 0, -1])

X3 = matrix(ZZ, [
    [2,-3,4,0],
    [4,4,53,-2],
])

b3 = vector(ZZ,[-4,0])

# Code pour l'EXERCICE
def calculate_r(S):
    r = 0
    while r<min(S.nrows(), S.ncols()) and S[r,r]!=0:
        r += 1
    return r

def solve(X,b):
    S, L, C = X.smith_form()
    bb = L*b
    # caculate the rank of the Smith normal form matrix S
    r = calculate_r(S)
    # Check if there exists a solution to the system X*z=b
    for i in range(r):
        if bb[i]%S[i,i]!=0:
            # If any entry of bb is not divisible by the corresponding diagonal element of S, return no solution
            return (False, vector(X.base_ring(), 1), identity_matrix(X.base_ring(), 1))
    
    # Check if there are no nonzero entries in the bottom rows of bb
    for i in range(r, S.nrows()):
        if bb[i] != 0:
            # If any entry in the bottom rows of bb is nonzero, return no solution
            return (False, vector(X.base_ring(), 1), identity_matrix(X.base_ring(), 1))
        
    # If all checks pass, construct a particular solution z and the matrix H whose columns span the kernel
    return (True, sum([bb[i] / S[i, i] * C[:, i] for i in range(r)]), C[:, r:])


if solve(X1,b1)[0] :
    z1 = solve(X1,b1)[1]

    H1 = solve(X1,b1)[2]
else : 
    z1, H1 = "No solution", "No solution"

if solve(X2,b2)[0] :
    z2 = solve(X2,b2)[1]

    H2 = solve(X2,b2)[2]
else : 
    z2, H2 = "No solution", "No solution"
    
if solve(X3,b3)[0] :
    z3 = solve(X3,b3)[1][:3] #on retire la dernière ligne qui correspond a la variable ajoutée pour le modulo

    H3 = solve(X3,b3)[2][:3,:] # idem
else : 
    z3, H3 = "No solution", "No solution"

# # Affichage des resultats

print("Une solution particuliere de X1*z1 = b1 est")
print(z1)
print("les solutions du systeme homogene sont engendres par")
print(H1)
print("Une solution particuliere de X2*z2 = b2 est")
print(z2)
print("les solutions du systeme homogene sont engendrees par")
print(H2)
print("Une solution particuliere du systeme 3 est")
print(z3)
print("les solutions du systeme homogene sont engendres par")
print(H3)

# ****************************************************************************
# RESOLUTION DE SYSTEME LINEAIRE NON-HOMOGENE
# ****************************************************************************

Une solution particuliere de X1*z1 = b1 est
[-7]
[-6]
[ 4]
les solutions du systeme homogene sont engendres par
[-6]
[-5]
[ 4]
Une solution particuliere de X2*z2 = b2 est
No solution
les solutions du systeme homogene sont engendrees par
No solution
Une solution particuliere du systeme 3 est
[-136]
[ -68]
[  16]
les solutions du systeme homogene sont engendres par
[-16  35]
[ -8  18]
[  2  -4]


In [21]:
reset()
print("""\
# ****************************************************************************
# STRUCTURE DU QUOTIENT
# ****************************************************************************
""")

# Donnees de l'enonce de l'exercice
# A little problem

A = matrix(ZZ,[
              [ -630,   735,   0,   735, -630],
              [ 1275, -1485, -15, -1470, 1275],
              [  630,  -630,   0,  -630,  630]])

# Code pour l'EXERCICE

S, L, C = A.smith_form()
inv_L = 1/L 
reponse = ""
for i in range(A.rank()):
    reponse+=f" Z/{S[i,i]}Z *"
if min(A.ncols(),A.nrows())-A.rank()>0 :
    reponse+=f"Z^{min(A.ncols(),A.nrows())-A.rank()}"
if reponse[-1]=="*" :
    reponse = reponse[:-1]

# # Affichage des resultats

print("La structure de Z^3/N est")
print(reponse)

# ****************************************************************************
# STRUCTURE DU QUOTIENT
# ****************************************************************************

La structure de Z^3/N est
 Z/15Z * Z/105Z * Z/630Z 


In [29]:
reset()
print("""\
# ****************************************************************************
# FACTEURS INVARIANTS
# ****************************************************************************
""")

# Donnees de l'enonce de l'exercice

A = matrix(ZZ,[
              [ -630,   735,   0,   735, -630],
              [ 1275, -1485, -15, -1470, 1275],
              [  630,  -630,   0,  -630,  630]])


# Code pour l'EXERCICE

S, L, C = A.smith_form()
rang = min(A.nrows(), A.ncols()) - A.rank()
fact_inv = [S[i, i] for i in range(min(S.nrows(), S.ncols())) if S[i, i] != 0]
reponse = f"The set of exponents is lcm({invariant_factors})Z = {lcm(invariant_factors)}Z" if rank == 0 else "There are no exponents."

# # Affichage des resultats


print("Le rang de Z^3 / N est")
print(rang)
print("Les facteurs invariants sont")
print(fact_inv)
print("Exposants ?")
print(reponse)

# ****************************************************************************
# FACTEURS INVARIANTS
# ****************************************************************************

Le rang de Z^3 / N est
0
Les facteurs invariants sont
[15, 105, 630]
Exposants ?
There are no exponents.
